In [ ]:
# cspell: ignore kernelspec ipykernel codemirror ipython nbconvert pygments nbformat 

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
import python_simulator

In [ ]:
# header of auto_test_result_*.csv
header = [
    "param_name", # 0
    "param_val", # 1
    "max_abs_lateral_error", # 2
    "max_abs_vel_error", # 3
    # "max_abs_yaw_error", # 4
    # "max_abs_acc_error", # 5
    # "max_abs_steer_error" # 6
]


plot_param_list = [
    # [label, header_index, unit_for_plot, nominal_value]
    ["steer_bias", 2, "[deg]", 0.0] ,
    ["steer_rate_lim", 2, "[rad/s]", python_simulator.steer_rate_lim],
    ["vel_rate_lim", 3, "[m/ss]", python_simulator.vel_rate_lim],
    ["steer_dead_band", 2, "[rad]", python_simulator.steer_dead_band],
    ["wheel_base",2, "[m]", python_simulator.L],
    # ["adaptive_gear_ratio_coef", 2,"[-]", None],
    ["acc_time_delay",3, "[s]", python_simulator.acc_time_delay],
    ["steer_time_delay",2, "[s]", python_simulator.steer_time_delay],
    ["acc_time_constant",3, "[s]", python_simulator.acc_time_constant],
    ["steer_time_constant",2, "[s]", python_simulator.steer_time_constant],
    ["accel_map_scale",3, "[-]", None],
    ["acc_scaling",3, "[-]", 1],
    ["steer_scaling",2, "[-]", 1],
]
print(plot_param_list)
print("len(plot_param_list)",len(plot_param_list))


In [ ]:
fig = plt.figure(figsize=(30, 18), tight_layout=True)

ax_list = []
for i in range(len(plot_param_list)):
    ax_list.append(plt.subplot(4, 3, i+1))

# ff_data_result_csv = "auto_test_result_final_model_control_trained_with_data_collected_by_ff_control.csv"
nominal_data_result_csv = "auto_test_result_final_model_control_trained_with_data_collected_by_nominal_control.csv"

df_info_list = [ 
    {"file_name": "nominal/auto_test_result_nominal_model_control.csv", 
     "label_name": "nominal"},

    # {"file_name": "poly(selected) (data using FF)/"+ff_data_result_csv, 
    #  "label_name" : "poly(selected) (data using FF)"},    
    # # {"file_name": "poly(full) (data using FF)/"+ff_data_result_csv, 
    # #  "label_name" : "poly(full) (data using FF)"},    
    # {"file_name": "poly(selected)+NN (data using FF)/"+ff_data_result_csv, 
    #  "label_name" : "poly(selected)+NN (data using FF)"},    
    # # {"file_name": "poly(full)+NN (data using FF)/"ff_data_result_csv, 
    # #  "label_name" : "poly(full)+NN (data using FF)"},
    # {"file_name": "NN (data using FF)/"+ff_data_result_csv, 
    #  "label_name" : "NN (data using FF)"},
    
    {"file_name": "poly(selected) (nominal control data)/"+nominal_data_result_csv, 
     "label_name" : "poly(selected) (nominal control data)"},    
    # {"file_name": "poly(full) (nominal control data)/"+nominal_data_result_csv, 
    #  "label_name" : "poly(full) (nominal control data)"},
    {"file_name": "poly(selected)+NN (nominal control data)/"+nominal_data_result_csv, 
     "label_name" : "poly(selected)+NN (nominal control data)"},    
    # {"file_name": "poly(full)+NN (nominal control data)/"+nominal_data_result_csv, 
    #  "label_name" : "poly(full)+NN (nominal control data)"},
    {"file_name": "NN (nominal control data)/"+nominal_data_result_csv, 
     "label_name" : "NN (nominal control data)"},        
]

cmap = plt.get_cmap("tab10") 
for l in range(len(df_info_list)):
    df = pd.read_csv(df_info_list[l]["file_name"], header=None,sep=",")
    for i in range(len(plot_param_list)):
        index_bool = df[0].values==plot_param_list[i][0]
        if any(index_bool):
            x = df[1].values[index_bool]
            
            if plot_param_list[i][0]=="adaptive_gear_ratio_coef":
                x = np.array(range(len(x)))
            x = np.array([float(x[j]) for j in range(len(x))])
            if plot_param_list[i][0]=="steer_bias":
                x *= 180.0/np.pi
            
                
            y = df[plot_param_list[i][1]].values[index_bool]
            ax_list[i].plot(x,y,"o-",label=df_info_list[l]["label_name"],color=cmap(l))
        else:
            pass
            if i==0:
                ax_list[i].plot([0],[0],"o-",label=df_info_list[l]["label_name"],color=cmap(l))


for i in range(len(plot_param_list)):        
    #ax_list[i].set_title(header[plot_param_list[i][1]]+" / "+plot_param_list[i][0], fontsize=16)
    ax_list[i].set_xlabel(plot_param_list[i][0] + " " + plot_param_list[i][2],fontsize=20)
    ax_list[i].set_ylabel(header[plot_param_list[i][1]],fontsize=20)
    ax_list[i].grid()

    if plot_param_list[i][3] is not None:
        ax_list[i].plot([plot_param_list[i][3],plot_param_list[i][3]],[-0.1,2],"k--")
    
    # lateral_error
    if 2==plot_param_list[i][1]:
        ax_list[i].set_ylim([-0.01,1.0])
    # vel_error
    if 3==plot_param_list[i][1]:
        ax_list[i].set_ylim([-0.01,1.0])

    # if(y.max()>2.0):
            #     ax_list[i].set_ylim([-0.1,2.0])
    if i==0:
        ax_list[i].legend(fontsize=13)
plt.savefig("auto_test_result.png")
plt.show()